# Threading and Multi processing

#### You can run a  code in parallel to speed up operation


## *Process:A process is an instance of a program (e.g a Python Intepreter or FIREFOX browser)* 
## *Thread: An entity within a process. A process can have multiple threads*


###  Advantages of Processes

1) Takes advantage of multiple CPUs and cores


2) Separate memory space -> Memory is not shared between processes


3) Great for CPU-bound processing


4) New process is stated independently from others


5) Processes are interuptable / killable


6) One GIL (Global Interpreter Lock) for each process -> avoids GIL limitation

### Disadvantages of  processes 
- Heavyweight 
- Starting a process is a slower process than starting a thread
- More memory
- IPC (Inter -process communication) is more complicated



# Threads

### Threads: An entity within a process  that can be scheduled(also known as a lightweight process)
#### A process can spawn multiple threads

#### Advantages of threads
1.) All threads within a process share memory


2.) Lightweight 


3.) Starting a thread is faster than starting a process


4.) Great for I/O bound processes

#### Disadvantages of threads
1.) Threading is limited by GIL: Only one thread at a time
    
    
2.) No effect for CPU bound tasks


3.) Not interruptable/ killable


4.) Careful with race conditions(When two or more threads want to modify the same variable at the same time) bugs or crushes

## GIL Global Interpreter Lock

#### Properties
1.) A lock that allows only one thread at a time to execute in Python


2.) Needed in CPython because memory management  is not a thread-safe

3.) CPython is the referrence Python implementation you get when you download and installPython from Python.org. In CPython 
there is a memory management that is not thread-safe. So in CPython there is a technique called referrence counting for memory management.



This means that objects created in Python have a referrence count variable that keeps track of the number of referrences that point to the object. So when the number reaches zero the memory occupied by this object can be released.



The problem is that these variables need protection from threads that can increase or decrease the value simultenously. When this happens it can cause memory leak that has not been released or it can incorrectly release memory when the object still exist.


This is the reason they introduced the GIL. you can use a couple of ways to avoid the GIL should you need multi-processing.



#### Avoiding GIL:
1.)  Use multiprocessing


2.)  Use a different, free threaded Python Implementation (Jython, IronPython) instead of CPython.


3.) Use Python as a wrapper for third-party libraries (C/C++) -> This is why numpy and scipy can be faster. NumPy, Scipy just use python wrapper while running in those faster languages.


In [9]:
from multiprocessing import Process
import os
import time


def square_numbers():
    for i in range(100):
        i*i
        time.sleep(0.1)
processes = []
# a good count is the number of cores on your machine

num_processors = os.cpu_count()
print(num_processors)

for i in range(num_processors):
    p =Process(target=square_numbers)
    processes.append(p)
    
    
# starting a process
for p in processes:
    p.start()

# join processes
for p in processes:
    p.join()
    
print('end main')
        
# the stuff above is to wait for the processes to finish while blocking the main thread

4
end main


## Threading API

In [11]:
from threading import Thread


num_threads = 10
threads = []

for i in range(num_threads):
    t = Thread(target=square_numbers)
    threads.append(t)

# to start threads
for t in threads:
    t.start()

# join threads: wait for them to complete by blocking the main thread until the thread is complete
for t in threads:
    t.join()
    

#### Since threads live in the same memory space they have access to the same data. This gives them an opportunity to share data

In [12]:
from threading import Thread
import os
import time


database_value = 0
def square_numbers():
    for i in range(100):
        i*i
        time.sleep(0.1)
threads = []


for i in range(num_processors):
    t = Thread(target=square_numbers)
    threads.append(t)
    
    
# start each threa
for t in threads:
    t.start()

# join threads: wait for them to complete
for t in threads:
    t.join()
    
print('end main')
        
# the stuff above is to wait for the processes to finish while blocking the main thread

end main


In [13]:
from threading import Thread
import time


# This should simulate a database as we share memory
database_value = 0

def increase():
    global database_value
    
    local_copy = database_value
    
    local_copy +=1
    # in order to wait a bit 0.1 seconds
    '''
    The end value is one instead of two because the first thread went on sleep and the second one immediately 
    started. This was all before the localupdated variable could change database so they all return to database
    value with an update of one.
    
    To prevent this you can use the lock.
    '''
    time.sleep(0.1)
    database_value = local_copy

print('Start value', database_value)

thread_1 = Thread(target=increase)
thread_2 = Thread(target=increase)

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

print('end value', database_value)

Start value 0
end value 1


Now using a lock

In [15]:
from threading import Thread, Lock
import time


# This should simulate a database as we share memory
database_value = 0

def increase(lock):
    global database_value
    
    with lock:
        local_copy = database_value

        local_copy +=1
        # in order to wait a bit 0.1 seconds
        '''
        Whenever you use lock.acquire() a lock you should release it below with a lock.release() otherwise it will remain stuck.
        The purpose of the lock is to allow the first thread to lock the memory and modify before releasing.

        It is however recommended to use with lock (Using lock as a context manager)
        '''
        time.sleep(0.1)
        database_value = local_copy
        



lock = Lock()
print('Start value', database_value)

thread_1 = Thread(target=increase, args=(lock,))
thread_2 = Thread(target=increase, args=(lock,))

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

print('end value', database_value)

Start value 0
end value 2


#### Now moving on to queue

In [ ]:
from threading import Thread, Lock
import time
from queue import Queue
#A queue is a linear data structure that follows a FIFO (First in first Out)



q = Queue()
q.put(1)
q.put(2)
q.put(3)

first =q.get()
print(first)
# To block the main thread until all the threads are done you use q.join()
q.join()


# When done with a queue method you invoke q.task_done()
q.task_done()


1


In [2]:
from threading import Thread, Lock, current_thread
import time
from queue import Queue


def worker(q, lock):
    while True:
        value = q.get()
        
        # do some processing
        with lock:
            print(f'In {current_thread().name} got {value}')
        q.task_done()
        
        
        
q = Queue()
lock = Lock()
num_threads = 10

for i in range(num_threads):
    thread = Thread(target=worker, args =(q, lock))
    thread.daemon = True
    thread.start()
    
for i in range(1, 21):
    q.put(i)
    

q.join()
print('end main')
# A daemon thread is a thread which dies when the main thread dies.
#If you do not use a daemon thread then our programme continues in the infinite while True loop

In Thread-16 got 1
In Thread-20 got 2
In Thread-19 got 3
In Thread-21 got 4
In Thread-17 got 5
In Thread-22 got 6
In Thread-18 got 7
In Thread-23 got 8
In Thread-24 got 9
In Thread-25 got 10
In Thread-16 got 11
In Thread-20 got 12
In Thread-19 got 13
In Thread-21 got 14
In Thread-17 got 15
In Thread-22 got 16
In Thread-18 got 17
In Thread-23 got 18
In Thread-24 got 19
In Thread-25 got 20
end main
